In [1]:
from redisvl.extensions.router.routes import Route, RoutingConfig

In [2]:
from redisvl.extensions.router.routes import Route

# Define individual routes manually with metadata
politics = Route(
    name="politics",
    references=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions"
    ],
    metadata={"priority": 1}
)

chitchat = Route(
    name="chitchat",
    references=[
        "hello",
        "how's the weather today?",
        "how are things going?"
    ],
    metadata={"priority": 2}
)

In [3]:
routes = [politics, chitchat]

In [4]:
config = RoutingConfig(max_k=2, distance_threshold=1.0, accumulation_method="avg")

In [5]:
import redis
from redisvl.extensions.router.semantic import SemanticRouter

# Create SemanticRouter named "topic-router"
redis_client = redis.Redis()
routes = [politics, chitchat]
topic_router = SemanticRouter(
    name="topic-router",
    routes=routes,
    routing_config=config,
    redis_client=redis_client,
    overwrite=True
)

13:38:58 redisvl.index.index INFO   Index already exists, overwriting.


In [6]:
topic_router("I am thinking about running for Governor in the state of VA. What do I need to consider?")

[{'name': 'chitchat',
  'references': ['hello', "how's the weather today?", 'how are things going?'],
  'metadata': {'priority': '2'},
  'score': '0.0641258955002',
  'avg_score': '0.0481971502304'},
 {'name': 'politics',
  'references': ["isn't politics the best thing ever",
   "why don't you tell me about your political opinions"],
  'metadata': {'priority': '1'},
  'score': '0.298070549965',
  'avg_score': '0.207850039005'}]

In [7]:
topic_router("don't you love politics?")

[{'name': 'chitchat',
  'references': ['hello', "how's the weather today?", 'how are things going?'],
  'metadata': {'priority': '2'},
  'score': '0.12840616703',
  'avg_score': '0.112765411536'},
 {'name': 'politics',
  'references': ["isn't politics the best thing ever",
   "why don't you tell me about your political opinions"],
  'metadata': {'priority': '1'},
  'score': '0.764727830887',
  'avg_score': '0.617416262627'}]

In [8]:
topic_router("how are you")

[{'name': 'chitchat',
  'references': ['hello', "how's the weather today?", 'how are things going?'],
  'metadata': {'priority': '2'},
  'score': '0.54086458683',
  'avg_score': '0.464291175207'},
 {'name': 'politics',
  'references': ["isn't politics the best thing ever",
   "why don't you tell me about your political opinions"],
  'metadata': {'priority': '1'},
  'score': '0.156601548195',
  'avg_score': '0.121711879969'}]

Need to work on a way to properly handle the "sum" strategy... Adding route references by vector distance when 0 is "more similar" means we would have to flip polarity in order to capture the right signal.

Avg is a safer strategy in general.. but assumes that the variance in the amount of references per route is not too high.

In [9]:
config = RoutingConfig(distance_threshold=0.1, accumulation_method="avg")

topic_router.update_routing_config(config)

In [10]:
# distance threshold is too low
topic_router("hello world")

[{'name': 'chitchat',
  'references': ['hello', "how's the weather today?", 'how are things going?'],
  'metadata': {'priority': '2'},
  'score': '0.756013274193',
  'avg_score': '0.423087894917'},
 {'name': 'politics',
  'references': ["isn't politics the best thing ever",
   "why don't you tell me about your political opinions"],
  'metadata': {'priority': '1'},
  'score': '0.175542235374',
  'avg_score': '0.138914197683'}]

In [11]:
topic_router("hello world", distance_threshold=0.3)

[{'name': 'chitchat',
  'references': ['hello', "how's the weather today?", 'how are things going?'],
  'metadata': {'priority': '2'},
  'score': '0.756013274193',
  'avg_score': '0.423087894917'},
 {'name': 'politics',
  'references': ["isn't politics the best thing ever",
   "why don't you tell me about your political opinions"],
  'metadata': {'priority': '1'},
  'score': '0.175542235374',
  'avg_score': '0.138914197683'}]